In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

import string


In [2]:
df = pd.read_csv('steam_reviews.csv')

# group the data by app_id
grouped_data = df.groupby('app_id')

# create an empty dataframe to store the selected data
selected_data = pd.DataFrame()

# loop through each group and select the first 30 rows
for name, group in grouped_data:
    selected_group = group.head(30)
    selected_data = selected_data.append(selected_group)

# save the selected data to a new csv file
df=selected_data
print("Len: " + str(len(df)))

df.fillna(value=0, inplace=True)

Len: 9450


# Small dataset

In [3]:
df.head()

,Unnamed: 0,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
469395,469395,70,Half-Life,84948211,english,the platforming makes me want to eat sand\n,1611014751,1611014751,False,0,...,True,False,False,76561198352346016,120,2,591.0,167.0,530.0,1.611017e+09
469396,469396,70,Half-Life,84947859,english,"Really fun, great story an crap.",1611014064,1611014064,True,0,...,True,False,False,76561198074088375,184,11,261.0,283.0,237.0,1.611015e+09
469397,469397,70,Half-Life,84946698,czech,nice\n,1611012114,1611012114,True,0,...,True,False,False,76561198960746554,3,2,5608.0,52.0,5555.0,1.611015e+09
469398,469398,70,Half-Life,84946092,english,good,1611011237,1611011237,True,0,...,True,False,False,76561198945746887,36,16,3297.0,2811.0,3297.0,1.611000e+09
469399,469399,70,Half-Life,84935389,english,"hey, sorry i saw your profile and i just thoug...",1611009975,1611009975,True,0,...,True,False,False,76561198911822629,61,26,1015.0,0.0,1015.0,1.587937e+09


In [4]:
# Output all columns name
list(df.columns)

['Unnamed: 0',
 'app_id',
 'app_name',
 'review_id',
 'language',
 'review',
 'timestamp_created',
 'timestamp_updated',
 'recommended',
 'votes_helpful',
 'votes_funny',
 'weighted_vote_score',
 'comment_count',
 'steam_purchase',
 'received_for_free',
 'written_during_early_access',
 'author.steamid',
 'author.num_games_owned',
 'author.num_reviews',
 'author.playtime_forever',
 'author.playtime_last_two_weeks',
 'author.playtime_at_review',
 'author.last_played']

In [5]:
# check if each game has approximate 30 reviews
df_sorted=df.groupby('app_name')['review'].count().sort_values(ascending=False)
df_sorted = df_sorted.reset_index(name='count')
df_sorted

,app_name,count
0,20XX,30
1,Rules Of Survival,30
2,STAR WARS™ Empire at War: Gold Pack,30
3,SOULCALIBUR VI,30
4,SOS,30
...,...,...
310,Finding Paradise,30
311,Farming Simulator 19,30
312,Farm Together,30
313,Farm Manager 2018,30


# Data Preprocessing

using nltk to elminate any distraction

In [6]:
# New imports 
import re
import nltk.corpus
from unidecode                        import unidecode
from nltk.tokenize                    import word_tokenize
from nltk                             import SnowballStemmer
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.preprocessing            import normalize

In [7]:
Columns = ['app_name','language','review']
df_target = df.loc[:, Columns]
df_target.head()

,app_name,language,review
469395,Half-Life,english,the platforming makes me want to eat sand\n
469396,Half-Life,english,"Really fun, great story an crap."
469397,Half-Life,czech,nice\n
469398,Half-Life,english,good
469399,Half-Life,english,"hey, sorry i saw your profile and i just thoug..."


In [8]:
# We no longer need text columns 
df = df.drop(columns=Columns)
df.head()

,Unnamed: 0,app_id,review_id,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
469395,469395,70,84948211,1611014751,1611014751,False,0,0,0.000000,0,True,False,False,76561198352346016,120,2,591.0,167.0,530.0,1.611017e+09
469396,469396,70,84947859,1611014064,1611014064,True,0,0,0.000000,0,True,False,False,76561198074088375,184,11,261.0,283.0,237.0,1.611015e+09
469397,469397,70,84946698,1611012114,1611012114,True,0,0,0.000000,0,True,False,False,76561198960746554,3,2,5608.0,52.0,5555.0,1.611015e+09
469398,469398,70,84946092,1611011237,1611011237,True,0,0,0.000000,0,True,False,False,76561198945746887,36,16,3297.0,2811.0,3297.0,1.611000e+09
469399,469399,70,84935389,1611009975,1611009975,True,0,0,0.478261,0,True,False,False,76561198911822629,61,26,1015.0,0.0,1015.0,1.587937e+09


In [9]:
scaler = MinMaxScaler()
df[list(df.columns)] = scaler.fit_transform(df[list(df.columns)])

In [10]:
df_target.head(60)

,app_name,language,review
469395,Half-Life,english,the platforming makes me want to eat sand\n
469396,Half-Life,english,"Really fun, great story an crap."
469397,Half-Life,czech,nice\n
469398,Half-Life,english,good
469399,Half-Life,english,"hey, sorry i saw your profile and i just thoug..."
469400,Half-Life,english,yes.
469401,Half-Life,english,The game is as good as people said it was. I w...
469402,Half-Life,english,"Amazing game. Its a timeless classic, would de..."
469403,Half-Life,english,make the scientists angry
469404,Half-Life,english,"""Where the fuck do I go"" The Game."



# TF-IDF
Build TF-IDF index in order to make text eligiable for k-means

statistical method for evaluating the importance of words in a document or corpus of documents. It is a way of measuring how relevant a word is to a document or set of documents by comparing the frequency of the word in the document to its frequency in the entire corpus.

# Heavy Review and App Name preprocess
* normalizing
* stemming
* tokenizing

Option to keep english only records, in order to have some coherence between data, but it is just an empirical hypotesis

In [11]:
# removes a list of words (ie. stopwords) from a tokenized list.
def removeWords(listOfTokens, listOfWords):
    return [token for token in listOfTokens if token not in listOfWords]

# applies stemming to a list of tokenized words
def applyStemming(listOfTokens, stemmer):
    return [stemmer.stem(token) for token in listOfTokens]

# removes any words composed of less than 2 or more than 21 letters
def twoLetters(listOfTokens):
    twoLetterWord = []
    for token in listOfTokens:
        if len(token) <= 2 or len(token) >= 21:
            twoLetterWord.append(token)
    return twoLetterWord

In [12]:
# Should we inlcude title in our analysis?
include_title = False

In [13]:
# some heavy preprocess
i = 0 
def process_row(row):
    global i 
    global include_title
    print("Tfidf Progress: "+str("{:.5f}".format(i/(len(df_target)-1) *100)) + "%",end="\r")
    i += 1
    
    language = row['language']
    reuslt = None
    if(include_title):
        result = (str(row['review']) + " " + str(row['app_name'])).replace(u'\ufffd', '8')   # Replaces the ASCII '�' symbol with '8'
    else:
        result = (str(row['review'])).replace(u'\ufffd', '8')   # Replaces the ASCII '�' symbol with '8'
    result = result.replace(',', '')          # Removes commas
    result = result.rstrip('\n')              # Removes line breaks
    result = result.casefold()                # Makes all letters lowercase

    result = re.sub('\W_',' ', result)        # removes specials characters and leaves only words
    result = re.sub("\S*\d\S*"," ", result)   # removes numbers and words concatenated with numbers IE h4ck3r. Removes road names such as BR-381.
    result = re.sub("\S*@\S*\s?"," ", result) # removes emails and mentions (words with @)
    result = re.sub(r'http\S+', '', result)   # removes URLs with http
    result = re.sub(r'www\S+', '', result)    # removes URLs with www
    
    result = re.sub(r"\s+(.)\1+\b", "",result)               # remove spammed chars
    result = ''.join([i for i in result if not i.isdigit()]) # remove numbers
    result = re.sub(r'[^\w\s]','',result)                    # remove punctuation
   

    listOfTokens = word_tokenize(result)
    twoLetterWord = twoLetters(listOfTokens)


    try: 
        # Not all languages are supported 
        # We remove stopwords and stem only with supported languages
        stopwords = nltk.corpus.stopwords.words(language)
        stopwords.append("game")
        
        param_stemmer = SnowballStemmer(language)
        listOfTokens = applyStemming(listOfTokens, param_stemmer)
        listOfTokens = removeWords(listOfTokens, stopwords)
        
        

    finally:
        # Output the tokens
        listOfTokens = list(filter(lambda item: len(item) < 25,listOfTokens))   # other spam filter
        listOfTokens = removeWords(listOfTokens, ["game","gam","igr","juego","you","jogao"]) # some game translations (en,en,ru,sp,ch,pt)
        
        result   = " ".join(listOfTokens)
        result = unidecode(result)
        result = re.sub(r'[^\x00-\x7F]+','', result)    # remove non ascii

        return result

In [14]:
english_only = False

In [15]:
if english_only:
    len_ = len(df_target)
    df_target = df_target[df_target.language == 'english']
    df_target.head()
    print("Length before filtering " + str(len_) + " Length after filtering "+ str(len(df_target)))

In [16]:
%%time
df_target['tokens'] = df_target.apply(lambda row: process_row(row),axis=1)

Wall time: 57 s 100.00000%


In [17]:
df_target.head(60)

,app_name,language,review,tokens
469395,Half-Life,english,the platforming makes me want to eat sand\n,the platforming makes me want to eat sand
469396,Half-Life,english,"Really fun, great story an crap.",really fun great story an crap
469397,Half-Life,czech,nice\n,nice
469398,Half-Life,english,good,good
469399,Half-Life,english,"hey, sorry i saw your profile and i just thoug...",hey sorry i saw your profile and i just though...
469400,Half-Life,english,yes.,yes
469401,Half-Life,english,The game is as good as people said it was. I w...,the is as good as people said it was i wasnt b...
469402,Half-Life,english,"Amazing game. Its a timeless classic, would de...",amazing its a timeless classic would definitel...
469403,Half-Life,english,make the scientists angry,make the scientists angry
469404,Half-Life,english,"""Where the fuck do I go"" The Game.",where the fuck do i go the


In [18]:
%%time
# tf-idf
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_target['tokens'])
tf_idf = pd.DataFrame(data = X.toarray(), columns=vectorizer.get_feature_names())

tf_idf.head()

Wall time: 1.11 s


,____,_udivitel,_z,aa,aaaa,aaaaa,aaaaaaaa,aaaaaaaaaaaany,aaaaaah,aaaaand,...,zwyklych,zwyklym,zy,zycia,zycie,zyciem,zyd,zydy,zyje,zywy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# One hot encoder
convert categorical data into a numerical representation that can be used in machine learning models.

In [19]:
%%time
bool_list = ['recommended','steam_purchase','received_for_free','written_during_early_access']
df[bool_list] = df[bool_list].astype(int)

Wall time: 17.8 ms


# Combine Data
Merge one hot and tfidf with some cleanup

In [20]:
df = pd.concat([df, tf_idf], axis=1)

In [21]:
df

,Unnamed: 0,app_id,review_id,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,comment_count,...,zwyklych,zwyklym,zy,zycia,zycie,zyciem,zyd,zydy,zyje,zywy
0,0.000000e+00,0.226103,0.999022,0.999136,0.999136,1.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.609080e-08,0.226103,0.999012,0.999124,0.999124,1.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9.218159e-08,0.226103,0.999007,0.999119,0.999119,1.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.382724e-07,0.226103,0.998992,0.999103,0.999103,1.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.843632e-07,0.226103,0.998982,0.999092,0.999092,1.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21696297,9.999998e-01,0.423219,0.999490,0.999611,0.999611,1.0,0.000000,0.0,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21696298,9.999999e-01,0.423219,0.999490,0.999611,0.999611,1.0,0.030769,0.0,0.626697,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21696299,9.999999e-01,0.423219,0.999484,0.999606,0.999606,1.0,0.000000,0.0,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21696300,1.000000e+00,0.423219,0.999449,0.999576,0.999576,1.0,0.000000,0.0,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df.fillna(0, inplace=True)

In [23]:
df

,Unnamed: 0,app_id,review_id,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,weighted_vote_score,comment_count,...,zwyklych,zwyklym,zy,zycia,zycie,zyciem,zyd,zydy,zyje,zywy
0,0.000000e+00,0.226103,0.999022,0.999136,0.999136,1.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.609080e-08,0.226103,0.999012,0.999124,0.999124,1.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9.218159e-08,0.226103,0.999007,0.999119,0.999119,1.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.382724e-07,0.226103,0.998992,0.999103,0.999103,1.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.843632e-07,0.226103,0.998982,0.999092,0.999092,1.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21696297,9.999998e-01,0.423219,0.999490,0.999611,0.999611,1.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21696298,9.999999e-01,0.423219,0.999490,0.999611,0.999611,1.0,0.030769,0.0,0.626697,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21696299,9.999999e-01,0.423219,0.999484,0.999606,0.999606,1.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21696300,1.000000e+00,0.423219,0.999449,0.999576,0.999576,1.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
